In [1]:
import numpy as np
from collections import OrderedDict
import sys, os
sys.path.append('C:/Users/KimDongyoung/Desktop/Github/my_git/mygit/DEEPLEARNING/퍼셉트론/mnist.py')
import pickle
from mnist import load_mnist

### 단순한 계층 구현하기

In [2]:
# 곱셈 계층

class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self,x,y):  # 순전파
    self.x = x
    self.y = y
    out = x * y   # z = x * y

    return out
  
  def backward(self,dout): # 역전파, dout은 상류에서 넘어온 미분이다
    dx = dout * self.y   # 상류에서 넘어온 미분(dout)에 y를 곱한다
    dy = dout * self.x

    return dx, dy

In [3]:
# 덧셈 계층

class AddLayer:
  def __init__(self):
    pass # 덧셈 계층에서는 초기화가 필요 없다
  
  def forward(self,x,y): 
    out = x + y
    return out
  def backward(self,dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [4]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)  # apple_price = apple * apple_num = 200
price = mul_tax_layer.forward(apple_price, tax) # price = apple_price * tax = 220

print(price)

# 역전파

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice) # dout = 1, dapple_price = 1.1 = dout*tax, dtax = 200 = dout*apple_price
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # dapple = 2.2 = dapple_price*apple_num, dapple_num = 110 = dapple_price*apple

print(dapple, dapple_num, dtax)  # 2.2 110 200


220.00000000000003
2.2 110.00000000000001 200


In [5]:
apple = 100 # 사과가격
apple_num = 2 # 사과개수
orange = 150 # 오렌지가격
orange_num = 3 # 오렌지개수
tax = 1.1 # 소비세

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple_num, apple) # 사과가격 * 사과개수
orange_price = mul_orange_layer.forward(orange_num, orange) # 오렌지가격 * 오렌지개수
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # 사과가격 + 오렌지가격
price = mul_tax_layer.forward(all_price, tax) # 소비세를 포함한 총 가격

# 역전파

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # 소비세 계층 역전파
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # 사과와 오렌지를 더한 가격 계층 역전파
dapple_num, dapple = mul_apple_layer.backward(dapple_price)
dorange_num, dorange = mul_orange_layer.backward(dorange_price)

print(price) # 715
print(dapple_num, dapple, dorange_num, dorange, dtax) # 2.2 110 3.3 165 650

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650


### 활성화 함수(Relu, Sigmoid) 계층 구현

In [6]:
# 순전파와 역전파를 포함한 ReLU 계층 구현

class Relu: 
  def __init__(self):
    self.mask = None # mask는 True/False로 구성된 넘파이 배열, 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외(0보다 큰 원소)는 False로 유지
  
  # 순전파
  def forward(self,x):
    self.mask = (x <= 0) # x의 원소 값이 0 이하인 인덱스를 True로 설정
    out = x.copy() # 입력 x를 복사하여 out에 저장
    out[self.mask] = 0 # mask의 원소가 True인 인덱스에 대응하는 원소를 0으로 설정, 즉 x가 0보다 작으면 0으로 변환, 그 외는 그대로 유지
    
    return out
  
  # 역전파
  def backward(self,dout):
    dout[self.mask] = 0
    dx = dout # 순전파 때의 입력인 x가 0보다 작으면 역전파 때의 값은 0, 그 외는 상류 값을 그대로 전달한다. 
    
    return dx

In [7]:
# 순전파와 역전파를 포함한 Sigmoid 계층 구현

class Sigmoid:
  def __init__(self):
    self.out = None
  
  def forward(self,x):
    out = 1 / 1 + np.exp(-x)  # y = 1 / 1 + exp(-x)
    self.out = out
    
    return out
  
  def backward(self,dout):
    dx = dout * (1.0 - self.out) * self.out   # y = 1 / 1 + exp(-x)의 미분은 y(1-y)이다
    
    return dx

### Affine, Softmax 계층 구현 

In [8]:
# 신경망의 순전파 때 수행하는 행렬의 내적을 기하학에서는 어파인 변환(Affine Transformation)이라고 불러서 Affine 계층이라는 이름을 사용한다.
# Affine 계층은 가중치 신호의 총합을 계산하고, 편향을 더한다.

# N개의 데이터를 묶어 순전파와 역전파를 수행한 Affine 계층 구현
class Affine: 
  def __init__(self,w,b): # 이 두 매개변수는 가중치와 편향을 의미하며 신경망이 학습될 시 갱신된다
    self.w = w  # 가중치
    self.b = b  # 편향
    self.x = None # 입력
    self.dw = None # 가중치의 미분
    self.db = None # 편향의 미분
   
  # 순전파  
  def forward(self, x):
    self.x = x
    out = np.dot(x,self.w) + self.b  # y = WX + B
    
    return out
   
  # 역전파  
  def backward(self,dout):
    dx = np.dot(dout,self.w.T)  # dx = dL/dy * W^T, self.w.T는 self.w의 전치행렬, 입력 x에 대한 상류 값의 미분으로 이전 층으로 전파해야 할 값이다
    self.dw = np.dot(self.x.T,dout) # dw = X^T * dL/dy, self.x.T는 self.x의 전치행렬
    self.db = np.sum(dout, axis = 0) # db = dL/dy의 각 원소의 총합이다, axis = 0은 행 방향으로 더한다
    
    return dx

In [9]:
def softmax(x):
    if x.ndim == 2:
        x = x - np.max(x, axis=1, keepdims=True) # 오버플로 대책
        x = np.exp(x)
        x /= np.sum(x, axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x) # 오버플로 대책
        x = np.exp(x) / np.sum(np.exp(x))
    return x
  
  # 평균 손실함수
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # t는 실제 정답 레이블
        y = y.reshape(1, y.size) # y는 신경망의 출력

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size  # np.log(0)이 -inf가 되는 것을 방지하기 위해 아주 작은 delta(1e-7)를 더해줌

In [10]:
# Softmax-with-Loss 계층

class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None # 손실
    self.y = None # softmax의 출력
    self.t = None # 정답 레이블(원-핫 인코딩 형태)
    
  def forward(self,x,t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y,self.t)
    
    # 순전파의 결과인 손실을 반환한다
    return self.loss
  
  def backward(self,dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size # 역전파의 결과로는 데이터의 개수로 나눠서 데이터 1개당 오차를 앞 계층으로 전파한다
    
    return dx

### 2층 신경망 TwoLayerNet 계층 구현  

In [11]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    # 가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 가중치 개수 = input_size x hidden_size
    self.params['b1'] = np.zeros(hidden_size) # 편향 개수 = hidden_size
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) # 가중치 개수 = hidden_size x output_size
    self.params['b2'] = np.zeros(output_size) # 편향 개수 = output_size
    
    # 신경망 계층 보관 (Affine 계층과 ReLU 계층)
    # 순서가 있는 딕셔너리를 사용하여 계층을 보관한다
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    
    # 마지막 계층, Softmax-with-Loss 계층
    self.lastLayer = SoftmaxWithLoss()
    
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x) # Affine 계층의 forward() 메서드와 ReLU 계층의 forward() 메서드를 호출한다 -> 순전파를 수행한다
      
    return x # 순전파의 결과를 반환한다. Affine 계층과 ReLU 계층을 통과한 결과가 마지막 Affine 계층을 통과하면서 예측이 완료된다
  
  # x: 입력 데이터, t: 정답 레이블
  # 손실 함수의 값이 반환된다
  def loss(self, x, t):
    y = self.predict(x) # predict() 메서드를 호출하여 예측을 수행한다. predict() 메서드에서 반환된 x값이 y에 저장된다, y는 예측 결과이다
    return self.lastLayer.forward(y, t) # self.lastLayer.forward()는 Softmax-with-Loss 계층의 forward() 메서드를 호출한다. 이 메서드는 손실을 반환한다
  
  def accuracy(self, x, t):
    y = self.predict(x) # 예측 결과
    y = np.argmax(y, axis = 1) # 예측 결과 중 가장 큰 값의 인덱스를 반환한다
    if t.ndim != 1: # 정답 레이블이 원-핫 인코딩 형태일 때, 정답 레이블을 1차원 배열로 변환한다
      t = np.argmax(t, axis = 1) # 정답 레이블 중 가장 큰 값의 인덱스를 반환한다. 형태의 원-핫 인코딩에서 1의 인덱스를 반환한다
      
    accuracy = np.sum(y == t) / float(x.shape[0]) # 정확도를 계산한다. 정확도는 (예측 결과와 정답 레이블이 같은 개수) / (데이터의 총 개수)로 계산한다
    return accuracy
  
  # 수치 미분을 사용하여 기울기를 계산하는 방법
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)
    
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    
    return grads
  
  # 오차역전파방법을 사용하여 기울기를 계산하는 방법
  def gradient(self, x, t):
    self.loss(x, t) # loss() 메서드를 호출하여 손실을 계산한다
    
    dout = 1
    dout = self.lastLayer.backward(dout) # self.lastLayer.backward()를 호출하여 역전파를 수행한다 -> softmax-with-loss 계층의 역전파를 수행한다
    
    layers = list(self.layers.values()) # layers에 순서가 있는 딕셔너리인 self.layers의 값을 리스트로 변환하여 저장한다
    layers.reverse() # layers의 순서를 뒤집는다
    for layer in layers: # layers의 각 원소인 layer에 대해 반복한다. layers는 Affine 계층과 ReLU 계층이다. 역으로 Affine2 -> ReLU1 -> Affine1 순서로 진행된다. 
      dout = layer.backward(dout) # layer.backward()를 호출하여 역전파를 수행한다. -> Affine 계층과 ReLU 계층의 역전파를 수행한다
     
    # 결과 저장  
    grads = {} # 기울기를 저장할 딕셔너리
    grads['W1'] = self.layers['Affine1'].dw # self.layers['Affine1'].dw에는 가중치 W1에 대한 기울기가 저장된다. Affine 계층의 self.dw에 저장된다
    grads['b1'] = self.layers['Affine1'].db # self.layers['Affine1'].db에는 편향 b1에 대한 기울기가 저장된다. Affine 계층의 self.db에 저장된다
    grads['W2'] = self.layers['Affine2'].dw # self.layers['Affine2'].dw에는 가중치 W2에 대한 기울기가 저장된다. Affine 계층의 self.dw에 저장된다
    grads['b2'] = self.layers['Affine2'].db
    print(grads['b1'])
    print('-'*50)
    print(grads['b2'])
    print('-'*50)
    print(grads['W1'])
    print('-'*50)
    print(grads['W2'])
    print('-'*50)    

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

# 미니배치 학습 구현
x_batch = x_train[:3]
t_batch = t_train[:3]

get_backpropagation = network.gradient(x_batch, t_batch)

[-0.00272464  0.00535511 -0.00327553 -0.00491696  0.          0.
  0.00538231 -0.00699602  0.00200531  0.00493462  0.00485425 -0.00210157
 -0.00345913 -0.00398161 -0.00357317  0.00887581 -0.00216292  0.
 -0.003773    0.00531618  0.00484305  0.00267067  0.         -0.00473203
  0.0011657   0.00017331 -0.00071017  0.00587813 -0.00658407  0.00502697
  0.00300069  0.          0.          0.00296616  0.00081108 -0.00629369
  0.00299486  0.          0.00613982  0.00277221  0.00046128  0.00202868
  0.00012037 -0.00165971 -0.000817   -0.00203699  0.00026662 -0.00194398
  0.          0.        ]
--------------------------------------------------
[-0.23368645  0.09995839  0.09960928  0.10009405 -0.23370426 -0.23287962
  0.10016377  0.10045478  0.10023242  0.09975763]
--------------------------------------------------
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
-------------------------

: 